# Вебинар 4. Домашнее задание  
Само домашнее задание нахожится в конце ноутбука

In [1]:
#!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path_1 = '/src'
module_path_2 = '/'
if module_path_1 not in sys.path:
    sys.path.append(module_path_1)
if module_path_2 not in sys.path:
    sys.path.append(module_path_2)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
n_items_before = data_train['item_id'].nunique()
n_users_before = data_train['user_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
n_users_after = data_train['user_id'].nunique()

print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))
print('Decreased # users from {} to {}'.format(n_users_before, n_users_after))

Prefilter items...

/home/alex/Рабочий стол/Rec_Sys/src/utils.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / data['quantity']


Data reduced from: 86865 to: 5000 samples...Done
Decreased # items from 86865 to 5000
Decreased # users from 2499 to 2478


# Домашнее задание  

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже (задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются



In [7]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    data : pd.DataFrame
    item_features : pd.DataFrame
        данные по продажам и данные по признакам
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
        self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        self.popularity_matrix = self.prepare_popularity_item_matrix(data, self.model)
        self.top_popular_items = self.popularity_recommendation(data)
        
     
    @staticmethod
    def prepare_matrix(data):
        
        # your_code
        user_item_matrix = pd.pivot_table(data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробоват ьдругие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                        )

        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix, K=1):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=K, num_threads=1)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=1, use_gpu=False):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads,
                                             use_gpu=use_gpu)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    
    def prepare_popularity_item_matrix(self, data, model, N=20):
        """Подготавливает матрицу похожести N популярных у каждого юзера"""

        popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)

        popularity = popularity[popularity['item_id'] != 999999]

        popularity = popularity.groupby('user_id').head(N)

        popularity.sort_values('user_id', ascending=False, inplace=True)

        popularity['similar_item'] = popularity['item_id'].apply(lambda x: self.id_to_itemid[self.model.similar_items(self.itemid_to_id[x], N=2)[1][0]])

        popularity = popularity.groupby('user_id')['similar_item'].unique()

        return popularity

    @staticmethod
    def popularity_recommendation(data, n=50):
      """Топ-n популярных товаров"""
      
      popular = data.groupby('item_id')['sales_value'].sum().reset_index()
      popular.sort_values('sales_value', ascending=False, inplace=True)
      
      recs = popular.head(n).item_id
      
      return recs.tolist()[1:]

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        
        res = list(self.popularity_matrix[user][:N].tolist())
        if len(res) < N:              # может случиться такая ситуация, когда юзер купил товаров меньше чем N, тогда его топ будет соответственно тоже меньше N
          lacks = N - len(res)        # и в таком случае мы просто добиваем популярными товарами
          res.extend(self.top_popular_items[:lacks])


        assert len(res) == N, 'Количество рекомендаций != {}, а = {} для юзера {}'.format(N, len(res), user)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code
        sim_user = self.model.similar_users(self.userid_to_id[user], N=N+1)[1:]
        sim_user = np.array(sim_user)[:, 0].astype('int').tolist()

        res = []
        for j in sim_user[:N]:
          sim_user_top_item = self.own_recommender.recommend(j, csr_matrix(self.user_item_matrix).tocsr(), N=1)
          sim_user_top_item = self.id_to_itemid[sim_user_top_item[0][0]]
          res.append(sim_user_top_item)

        res = list(set(res))

        if len(res) < N:              # может случиться такая ситуация, когда в топе товаров у похожих юзеров находятся одинаковые товары
          lacks = N - len(res)        # и в таком случае мы просто добиваем популярными товарами
          res.extend(self.top_popular_items[:lacks])
        

        

        assert len(res) == N, 'Количество рекомендаций != {}, а = {} для юзера {}'.format(N, len(res), user)
        return res

In [8]:
%%time
main_recom = MainRecommender(data_train)


CPU times: user 25.6 s, sys: 18.6 s, total: 44.2 s
Wall time: 18.3 s


In [9]:
%%time
main_recom.get_similar_items_recommendation(16)

CPU times: user 63 µs, sys: 39 µs, total: 102 µs
Wall time: 106 µs


[1016260, 861792, 904493, 889551, 1022097]

In [10]:
%%time
main_recom.get_similar_users_recommendation(3)

CPU times: user 27.7 ms, sys: 45 ms, total: 72.7 ms
Wall time: 22.3 ms


[949257, 1048237, 1036753, 1067443, 1029112]

In [11]:
res = main_recom.popularity_matrix.reset_index()
res.head()

,user_id,similar_item
0,1,"[8090560, 6554400, 1077540, 952163, 1096290, 8..."
1,2,"[5569845, 9707498, 948670, 988693, 951674, 878..."
2,3,"[1076875, 9189814, 945998, 839419, 12132315, 9..."
3,4,"[823721, 1021324, 937791, 15926844, 884518, 89..."
4,5,"[971922, 9245106, 1126252, 1035691, 937791, 94..."


In [12]:
res['top5_sim_items'] = res['user_id'].apply(lambda x: main_recom.get_similar_items_recommendation(x))

In [13]:
res.head()

,user_id,similar_item,top5_sim_items
0,1,"[8090560, 6554400, 1077540, 952163, 1096290, 8...","[8090560, 6554400, 1077540, 952163, 1096290]"
1,2,"[5569845, 9707498, 948670, 988693, 951674, 878...","[5569845, 9707498, 948670, 988693, 951674]"
2,3,"[1076875, 9189814, 945998, 839419, 12132315, 9...","[1076875, 9189814, 945998, 839419, 12132315]"
3,4,"[823721, 1021324, 937791, 15926844, 884518, 89...","[823721, 1021324, 937791, 15926844, 884518]"
4,5,"[971922, 9245106, 1126252, 1035691, 937791, 94...","[971922, 9245106, 1126252, 1035691, 937791]"


In [14]:
%%time
res['top5_sim_users'] = res['user_id'].apply(lambda x: main_recom.get_similar_users_recommendation(x))

CPU times: user 1min 20s, sys: 1min 21s, total: 2min 41s
Wall time: 44.9 s


In [15]:
res.head()

,user_id,similar_item,top5_sim_items,top5_sim_users
0,1,"[8090560, 6554400, 1077540, 952163, 1096290, 8...","[8090560, 6554400, 1077540, 952163, 1096290]","[949257, 1070803, 939860, 854133, 916122]"
1,2,"[5569845, 9707498, 948670, 988693, 951674, 878...","[5569845, 9707498, 948670, 988693, 951674]","[1117219, 9859017, 1036753, 1029112, 896666]"
2,3,"[1076875, 9189814, 945998, 839419, 12132315, 9...","[1076875, 9189814, 945998, 839419, 12132315]","[949257, 1048237, 1036753, 1067443, 1029112]"
3,4,"[823721, 1021324, 937791, 15926844, 884518, 89...","[823721, 1021324, 937791, 15926844, 884518]","[1117219, 1029112, 8019916, 1052920, 918046]"
4,5,"[971922, 9245106, 1126252, 1035691, 937791, 94...","[971922, 9245106, 1126252, 1035691, 937791]","[926465, 1036753, 1070803, 1067443, 1052920]"


Проверка, что все работает

In [16]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender as MR

In [17]:
mod = MR(data_train)

Экземпляр класса успешно создался из импортированного модуля